In [1]:
import pandas as pd
import datetime
import requests
from bs4 import BeautifulSoup
import time
import datetime
import pickle

In [3]:
with open('../data/merged_data_concat.pickle', 'rb') as f:
    base = pickle.load(f)

### KBS9시 뉴스시청률 크롤링

In [4]:
def get_date_string(start_date, period=365):
    # 20190721 형태의 Date String List 생성
    return [
        (start_date + datetime.timedelta(days=day)).strftime("%Y%m%d")
        for day in range(period)
    ]

In [5]:
def crawling(date):
    
    base_url = "https://www.nielsenkorea.co.kr/tv_terrestrial_day.asp?menu=Tit_1&sub_menu=1_1&area=00&begin_date="
    my_percent = []
    
    for i in date: 
        
        html = requests.post(base_url+i)
        #time.sleep()
        html.encoding = 'utf-8'
        soup = BeautifulSoup(html.text, 'html.parser')

        channel  = soup.find_all(class_='tb_txt_center')
        text = soup.find_all(class_= 'tb_txt')
        percent = soup.find_all(class_='percent')
        percent1 = soup.find_all(class_='percent_g')
        
        channel = channel[1::2]
        
        c=[]
        t=[]
        p=[]
        p1=[]
        
        for h in channel:
            c.append(h.text.strip())
        
        for j in text:
            t.append(j.text.strip())

        for k in percent:
            p.append(k.text.strip())

        for m in percent1:
            p1.append(m.text.strip())

        df = pd.DataFrame()
        df['channel'] = c[0:20]
        df['text'] = t[0:20]
        df['percent'] =p[0:10]+p1[0:10]
        
        try :
            index = df['text'].tolist().index('KBS9시뉴스')  # 9시 뉴스의  index 반환
            #print(df['percent'][index])
            #print(df['text'][index])
            my_percent.append(df['percent'][index])
        except:
            my_percent.append("")
            
    return my_percent

In [6]:
crawling_time = crawling(base['date'].unique())

In [7]:
crawling = pd.DataFrame()
date_list = base['date'].unique()
crawling['date'] = date_list
crawling['뉴스시청률']=crawling_time

In [8]:
base = base.merge(crawling, on=['date'],how='left')

In [9]:
base['뉴스시청률'] = base['뉴스시청률'].apply(lambda x: float(x))

In [10]:
with open('../data/merged_data_concat.pickle', 'wb') as f:
    pickle.dump(base, f)